<div align='center'>
    <h1>Sinusoidal Gratings Generator</h1>
    
    <h4>by Jason Sohn (<a href="https://jasonsohn.com">website</a>)</h4>
</div>

## Intro

This tool generates custom sinusoidal gratings for cognitive science research. No coding needed.

This demonstration closely follows the paper: *"Can categorical knowledge be used in visual search"* by S. Helie et al.

In effect, I have rewritten a small part of the [Psychophysics Toolbox by Brainard](http://cda.psych.uiuc.edu/matlab_class/Psychtoolbox%20-%20Introduction.htm), a MATLAB program released in 1998, in modern Python. I added interactivity and made it easier to use sinusoidal gratings with [PyTorch](https://github.com/pytorch/pytorch), the predominant deep learning research framework today.

For advanced users, I recommend trying out this demonstration first, then heading over to the [PyTorch-specific tutorial](https://hydo.ai). Also, feel free to browse the code [on GitHub](https://github.com/tensorturtle/deep-sinusoidal-grating)

In [1]:
# Limit some inputs if running on a tiny, weak computer
TINY_SERVER = True

In [2]:
import glob
import os
import pickle
import shutil
from pathlib import Path
import base64
import requests

In [3]:
from scipy import ndimage
import numpy as np
from PIL import Image, ImageDraw, ImageFilter
import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import interact, widgets
from ipyfilechooser import FileChooser
from IPython.display import HTML

In [4]:
from SineGratesDataset import SineGrates, pil_to_tensor, circular_sinegrate, show_img

In [13]:
# use proquint API to assign unique ids to each session
URL = 'http://unique.tensorturtle.com'
response = requests.get(URL)
session_proquint = response.text

## What is a sinusoidal grating?

The pixel luminance values in a sinusoidal grating image is defined according to 

$$ I(\mathbf{x}) = A \sin(\mathbf{\omega} \mathbf{x} + \rho) $$

where
+ $ A $ is the amplitude. For our use, it is fixed at 1.
+ $ \rho $ is the phase. This generator enables you to choose between 0, or a randommized value.
+ $ \mathbf{\omega} $ is the frequency, in units of 'cycles per degree (of human vision)'; one image is assumed to occupy 5 degrees of the visual field.

Sinusoidal gratings of various frequencies and orientations are used by S. Helie, et al. in *"Can categorical knowledge be used in visual search"* to explore the "effect of display size on perceptual categorization as a function of category structure".

## The three parameters that define our sinusoidal gratings

Play with the sliders!

*sidenote: Phase shift is not a parameter in the original paper, but was added during experiments with artificial neural networks. ([read more about it here](https://jasonsohn.com))*

If you do not see three sliders below, please wait or reload this page.

In [5]:
def demo_sinusoidal_grating(frequency, rotation, phase_shift):
    show_img(circular_sinegrate(frequency, rotation, phase_shift=phase_shift, image_size=(256,256)))
interact(demo_sinusoidal_grating, frequency=(1, 20, 0.1), rotation=(0, 180, 1), phase_shift=(0, 2*np.pi, 0.1))

interactive(children=(FloatSlider(value=10.0, description='frequency', max=20.0, min=1.0), IntSlider(value=90,…

<function __main__.demo_sinusoidal_grating(frequency, rotation, phase_shift)>

## Define a distribution from which to sample the parameters

In [6]:
scheme_w = widgets.ToggleButtons(
    options=['Rule-Based', 'Information-Integration'],
    description='Scheme:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    #tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

In [7]:
display(scheme_w)

ToggleButtons(description='Scheme:', options=('Rule-Based', 'Information-Integration'), value='Rule-Based')

In [8]:
int_w = widgets.IntText(
    value=200,
    description='Dataset size:',
    disabled=False
)

In [9]:
display(int_w)

IntText(value=200, description='Dataset size:')

In [10]:
bool_w = widgets.Checkbox(
    value=False,
    description='Randomize phase',
    disabled=False,
    indent=False
)

In [11]:
display(bool_w)

Checkbox(value=False, description='Randomize phase', indent=False)

In [21]:
Path('temp').mkdir(parents=True, exist_ok=True)

In [12]:
from IPython.display import display
button = widgets.Button(description="See/modify distribution")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    
        
    with output:
        if int_w.value > 4000 and TINY_SERVER:
            print("Oof, you're asking too much from this little server. To create datasets greater than 4000 images, please download this jupyter notebook and run it locally.")
        else:
            if scheme_w.value == 'Rule-Based':
                    cat_scheme = 'rb'
            elif scheme_w.value == 'Information-Integration':
                cat_scheme = 'ii'
            dataset_length = int_w.value // 2 # we need to split into 2, because condition 'a' and 'b' datasets are generated separately
            randomize_phase = bool_w.value
            rb_params = {
                    'a_means':[[30,50],[50,70]],
                    'b_means':[[50,30],[70,50]],
                    'a_covariances':[[[10,0],[0,150]],[[150,0],[0,10]]],
                    'b_covariances':[[[10,0],[0,150]],[[150,0],[0,10]]]
                }

            ii_params = {
                'a_means':[40,50],
                'b_means':[60,50],
                'a_covariances':[[10,0],[0,280]],
                'b_covariances':[[10,0],[0,280]]
            }


            if cat_scheme == 'rb':
                dist_params = rb_params
            elif cat_scheme == 'ii':
                dist_params = ii_params
            dataset = SineGrates(cat_scheme=cat_scheme, dist_params=dist_params, length=dataset_length, transform=None, randomize_phase=randomize_phase)
            def modify_dist_params_ii(**kwargs):
                new_params = {
                    'a_means':[kwargs['a_means_x'], kwargs['a_means_y']],
                    'b_means':[kwargs['b_means_x'], kwargs['b_means_y']],
                    'a_covariances':[[kwargs['a_cov_x'],0],[0,kwargs['a_cov_y']]],
                    'b_covariances':[[kwargs['b_cov_x'],0],[0,kwargs['b_cov_y']]]
                }
                dataset.set_dist_params(new_params)
                pickle.dump(dataset, open(f'temp/distribution_{session_proquint}.pkl', 'wb'))
                plt.show(dataset.plot_final())
            def modify_dist_params_rb(**kwargs):
                new_params = {
                    'a_means':[[kwargs['a_means_1_x'], kwargs['a_means_1_y']],
                               [kwargs['a_means_2_x'], kwargs['a_means_2_y']]],
                    'b_means':[[kwargs['b_means_1_x'], kwargs['b_means_1_y']],
                               [kwargs['b_means_2_x'], kwargs['b_means_2_y']]],
                    'a_covariances':[[[kwargs['a_cov_1_x'],0],[0,kwargs['a_cov_1_y']]],
                                     [[kwargs['a_cov_2_x'],0],[0,kwargs['a_cov_2_y']]]],
                    'b_covariances':[[[kwargs['b_cov_1_x'],0],[0,kwargs['b_cov_1_y']]],
                                     [[kwargs['b_cov_2_x'],0],[0,kwargs['b_cov_2_y']]]],
                }
                dataset.set_dist_params(new_params)
                pickle.dump(dataset, open(f'temp/distribution_{session_proquint}.pkl', 'wb'))
                plt.show(dataset.plot_final())
            if cat_scheme == 'ii':
                interact(modify_dist_params_ii, 
                         a_means_x = (0,80,1),
                         a_means_y = (0,100,1),
                         b_means_x = (0,120,1),
                         b_means_y = (0,100,1),
                         a_cov_x = (0,20,1),
                         a_cov_y = (0,560, 1),
                         b_cov_x = (0,20,1),
                         b_cov_y = (0,560, 1),
                        )
            elif cat_scheme == 'rb':
                    interact(modify_dist_params_rb, 
                             a_means_1_x = (0,60,1),
                             a_means_1_y = (0,100,1),
                             a_means_2_x = (0,100,1),
                             a_means_2_y = (0,140,1),
                             b_means_1_x = (0,100,1),
                             b_means_1_y = (0,60,1),
                             b_means_2_x = (0,140,1),
                             b_means_2_y = (0,100,1),
                             a_cov_1_x = (0,20,1),
                             a_cov_1_y = (0,300,1),
                             a_cov_2_x = (0,300,1),
                             a_cov_2_y = (0,20,1),
                             b_cov_1_x = (0,20,1),
                             b_cov_1_y = (0,300,1),
                             b_cov_2_x = (0,300,1),
                             b_cov_2_y = (0,20,1),
                        )
            print("")

button.on_click(on_button_clicked)



Button(description='See/modify distribution', style=ButtonStyle())

Output()

Next, generate the actual images:

In [14]:
def clean_dir(path, keep_last=10):
    files = glob.glob(str(Path(path))+'/*')
    asc = sorted(files, key=lambda t: os.stat(t).st_mtime)
    for i in range(len(asc)-keep_last):
        shutil.rmtree(asc[i])

In [15]:
def zip_session_dataset(in_dir, out_dir, session_proquint):
    to_zip = Path(in_dir)/session_proquint
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir/session_proquint
    shutil.make_archive(out_path, 'zip', to_zip)

In [16]:
button_2 = widgets.Button(description="Generate dataset")
output_2 = widgets.Output()

display(button_2, output_2)

def on_button_clicked_2(b):

    with output_2:
        print("Generating dataset...")
        dataset = pickle.load(open(f'temp/distribution_{session_proquint}.pkl', 'rb'))
        dataset.save_dataset(Path('output')/session_proquint)
        print(f"{len(dataset)} examples have been created. Download them now.")
        zip_session_dataset('output','zipped',session_proquint)
        
        # also delete old outputs 
        clean_dir('output', keep_last=50)
        clean_dir('zipped', keep_last=50)
        clean_dir('temp', keep_last=50)
        
        filename = f"zipped/{session_proquint}.zip"
        title = "Click here to download your dataset"
        edit_download_html(htmlWidget, filename, title = title, session_proquint=session_proquint)

button_2.on_click(on_button_clicked_2)

Button(description='Generate dataset', style=ButtonStyle())

Output()

In [17]:
def edit_download_html(htmlWidget, filename, title = "Click here to download:", session_proquint=''):
    
    # Change widget html temperarily to a font-awesome spinner
    htmlWidget.value = "<i class=\"fa fa-spinner fa-spin fa-2x fa-fw\"></i><span class=\"sr-only\">Loading...</span>"
    
    # Process raw data
    data = open(filename, "rb").read()
    b64 = base64.b64encode(data)
    payload = b64.decode()
    
    # Create and assign html to widget
    html = '<a download="{filename}" href="data:application/zip;base64,{payload}" target="_blank">{title}</a>'
    #htmlWidget.value = html.format(payload = payload, title = title+filename, filename = filename)
    htmlWidget.value = html.format(payload = payload, title = title, filename=f"sinusoidal-grating-{session_proquint}") # don't show filename

htmlWidget = ipywidgets.HTML(value = '')
htmlWidget

HTML(value='')

In [ ]:
print("Reload site to start again.")

In [19]:
print('\n'*30)

Copyright © 2022 Jason Sohn

Released under the MIT license.